# 25 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBClassifier

2023-11-30 05:19:48.171147: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 05:19:50.615587: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 05:19:50.615690: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 05:19:50.988227: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 05:19:51.740005: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 05:19:51.740999: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [2]:
no_of_patients=25

In [3]:
train=sorted(glob('files3/*.edf'))
train=train[:no_of_patients]
train

['files3/S001R05.edf',
 'files3/S002R05.edf',
 'files3/S003R05.edf',
 'files3/S004R05.edf',
 'files3/S005R05.edf',
 'files3/S006R05.edf',
 'files3/S007R05.edf',
 'files3/S008R05.edf',
 'files3/S009R05.edf',
 'files3/S010R05.edf',
 'files3/S011R05.edf',
 'files3/S012R05.edf',
 'files3/S013R05.edf',
 'files3/S014R05.edf',
 'files3/S015R05.edf',
 'files3/S016R05.edf',
 'files3/S017R05.edf',
 'files3/S018R05.edf',
 'files3/S019R05.edf',
 'files3/S020R05.edf',
 'files3/S021R05.edf',
 'files3/S022R05.edf',
 'files3/S023R05.edf',
 'files3/S024R05.edf',
 'files3/S025R05.edf']

In [4]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [5]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [6]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [7]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [8]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(300000, 37475, 300000, 37475)

In [9]:
print(xtest.iloc[:,-1].values)

[ 1.01e-04  1.30e-05 -1.50e-05 ...  6.00e-06  3.00e-06 -7.00e-06]


In [10]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,-0.000021,-0.000036,-0.000044,-0.000012,-0.000038,-0.000038,-0.000030,-0.000011,-0.000027,-0.000023,...,-0.000021,-0.000011,-0.000026,-0.000012,-0.000030,-0.000024,-0.000012,-0.000024,-0.000022,-0.000012
299996,-0.000033,-0.000040,-0.000046,-0.000014,-0.000040,-0.000039,-0.000030,-0.000012,-0.000029,-0.000027,...,-0.000017,-0.000011,-0.000025,-0.000010,-0.000024,-0.000009,-0.000011,-0.000018,-0.000017,-0.000009
299997,-0.000023,-0.000035,-0.000041,-0.000005,-0.000035,-0.000034,-0.000028,-0.000012,-0.000029,-0.000023,...,-0.000015,-0.000010,-0.000024,-0.000007,-0.000024,-0.000010,-0.000007,-0.000016,-0.000015,-0.000008
299998,-0.000020,-0.000024,-0.000032,0.000004,-0.000025,-0.000026,-0.000019,-0.000007,-0.000023,-0.000019,...,-0.000011,-0.000003,-0.000019,-0.000004,-0.000018,0.000000,0.000001,-0.000007,0.000002,0.000000


In [11]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [12]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [13]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_3400/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_3400/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_3400/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,-0.000021,-0.000036,-0.000044,-0.000012,-0.000038,-0.000038,-0.000030,-0.000011,-0.000027,-0.000023,...,-0.000021,-0.000011,-0.000026,-0.000012,-0.000030,-0.000024,-0.000012,-0.000024,-0.000022,-0.000012
299996,-0.000033,-0.000040,-0.000046,-0.000014,-0.000040,-0.000039,-0.000030,-0.000012,-0.000029,-0.000027,...,-0.000017,-0.000011,-0.000025,-0.000010,-0.000024,-0.000009,-0.000011,-0.000018,-0.000017,-0.000009
299997,-0.000023,-0.000035,-0.000041,-0.000005,-0.000035,-0.000034,-0.000028,-0.000012,-0.000029,-0.000023,...,-0.000015,-0.000010,-0.000024,-0.000007,-0.000024,-0.000010,-0.000007,-0.000016,-0.000015,-0.000008
299998,-0.000020,-0.000024,-0.000032,0.000004,-0.000025,-0.000026,-0.000019,-0.000007,-0.000023,-0.000019,...,-0.000011,-0.000003,-0.000019,-0.000004,-0.000018,0.000000,0.000001,-0.000007,0.000002,0.000000


In [14]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [15]:
model=XGBClassifier(n_estimators=500)
model.fit(x8,y8,early_stopping_rounds=10, eval_set=[(xv8, yv8)])
y_pred=model.predict(xt8)
print(classification_report(yt8,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.91089
[1]	validation_0-mlogloss:2.78557
[2]	validation_0-mlogloss:2.70545
[3]	validation_0-mlogloss:2.64570
[4]	validation_0-mlogloss:2.59139
[5]	validation_0-mlogloss:2.55158
[6]	validation_0-mlogloss:2.51291
[7]	validation_0-mlogloss:2.48394
[8]	validation_0-mlogloss:2.45068
[9]	validation_0-mlogloss:2.42893
[10]	validation_0-mlogloss:2.40568
[11]	validation_0-mlogloss:2.38613
[12]	validation_0-mlogloss:2.36814
[13]	validation_0-mlogloss:2.35298
[14]	validation_0-mlogloss:2.33874
[15]	validation_0-mlogloss:2.32130
[16]	validation_0-mlogloss:2.30208
[17]	validation_0-mlogloss:2.28768
[18]	validation_0-mlogloss:2.27454
[19]	validation_0-mlogloss:2.26282
[20]	validation_0-mlogloss:2.25096
[21]	validation_0-mlogloss:2.23579
[22]	validation_0-mlogloss:2.22535
[23]	validation_0-mlogloss:2.21720
[24]	validation_0-mlogloss:2.20857
[25]	validation_0-mlogloss:2.19888
[26]	validation_0-mlogloss:2.19166
[27]	validation_0-mlogloss:2.18132
[28]	validation_0-mlogloss:2.1

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
9375/9375 [==============================] - 9s 950us/step - loss: 1.8388 - val_loss: 2.0449
Epoch 2/10
9375/9375 [==============================] - 9s 938us/step - loss: 1.2869 - val_loss: 2.0205
Epoch 3/10
9375/9375 [==============================] - 9s 996us/step - loss: 1.1744 - val_loss: 2.0770
Epoch 4/10
9375/9375 [==============================] - 6s 648us/step - loss: 1.0994 - val_loss: 2.1018
Epoch 5/10
9375/9375 [==============================] - 6s 668us/step - loss: 1.0556 - val_loss: 2.3199
Epoch 6/10
9375/9375 [==============================] - 6s 669us/step - loss: 1.0297 - val_loss: 2.2398
Epoch 7/10
9375/9375 [==============================] - 6s 675us/step - loss: 1.0090 - val_loss: 2.3246
Epoch 8/10
9375/9375 [==============================] - 6s 665us/step - loss: 0.9954 - val_loss: 2.2565
Epoch 9/10
9375/9375 [==============================] - 6s 655us/step - loss: 0.9814 - val_loss: 2.4668
Epoch 10/10
9375/9375 [==============================] - 6s 666u

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

1172/1172 [==============================] - 0s 287us/step
              precision    recall  f1-score   support

           0       0.23      0.19      0.20      1499
           1       0.20      0.40      0.26      1499
           2       0.36      0.55      0.44      1499
           3       0.44      0.28      0.34      1499
           4       0.79      0.69      0.74      1499
           5       0.56      0.60      0.58      1499
           6       0.28      0.52      0.37      1499
           7       0.63      0.34      0.44      1499
           8       0.58      0.71      0.64      1499
           9       0.52      0.65      0.58      1499
          10       0.54      0.29      0.38      1499
          11       0.41      0.44      0.42      1499
          12       0.50      0.43      0.46      1499
          13       0.29      0.40      0.34      1499
          14       0.21      0.20      0.21      1499
          15       0.68      0.74      0.71      1499
          16       0.4

## 0-16

In [16]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [17]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_3400/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_3400/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_3400/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,-0.000021,-0.000036,-0.000044,-0.000012,-0.000038,-0.000038,-0.000030,-0.000011,-0.000027,-0.000023,...,-0.000021,-0.000011,-0.000026,-0.000012,-0.000030,-0.000024,-0.000012,-0.000024,-0.000022,-0.000012
299996,-0.000033,-0.000040,-0.000046,-0.000014,-0.000040,-0.000039,-0.000030,-0.000012,-0.000029,-0.000027,...,-0.000017,-0.000011,-0.000025,-0.000010,-0.000024,-0.000009,-0.000011,-0.000018,-0.000017,-0.000009
299997,-0.000023,-0.000035,-0.000041,-0.000005,-0.000035,-0.000034,-0.000028,-0.000012,-0.000029,-0.000023,...,-0.000015,-0.000010,-0.000024,-0.000007,-0.000024,-0.000010,-0.000007,-0.000016,-0.000015,-0.000008
299998,-0.000020,-0.000024,-0.000032,0.000004,-0.000025,-0.000026,-0.000019,-0.000007,-0.000023,-0.000019,...,-0.000011,-0.000003,-0.000019,-0.000004,-0.000018,0.000000,0.000001,-0.000007,0.000002,0.000000


In [18]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [19]:
model2=XGBClassifier(n_estimators=500)
model2.fit(x16,y16,early_stopping_rounds=10, eval_set=[(xv16, yv16)])
y_pred=model2.predict(xt16)
print(classification_report(yt16,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.85250
[1]	validation_0-mlogloss:2.71715
[2]	validation_0-mlogloss:2.61865
[3]	validation_0-mlogloss:2.53303
[4]	validation_0-mlogloss:2.46961
[5]	validation_0-mlogloss:2.41320
[6]	validation_0-mlogloss:2.36357
[7]	validation_0-mlogloss:2.32255
[8]	validation_0-mlogloss:2.28311
[9]	validation_0-mlogloss:2.24802
[10]	validation_0-mlogloss:2.21245
[11]	validation_0-mlogloss:2.18665
[12]	validation_0-mlogloss:2.16105
[13]	validation_0-mlogloss:2.13499
[14]	validation_0-mlogloss:2.11104
[15]	validation_0-mlogloss:2.08618
[16]	validation_0-mlogloss:2.06465
[17]	validation_0-mlogloss:2.04719
[18]	validation_0-mlogloss:2.02764
[19]	validation_0-mlogloss:2.01236
[20]	validation_0-mlogloss:1.99196
[21]	validation_0-mlogloss:1.97649
[22]	validation_0-mlogloss:1.95769
[23]	validation_0-mlogloss:1.94225
[24]	validation_0-mlogloss:1.92823
[25]	validation_0-mlogloss:1.91229
[26]	validation_0-mlogloss:1.89965
[27]	validation_0-mlogloss:1.88623
[28]	validation_0-mlogloss:1.8

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10
9375/9375 [==============================] - 6s 674us/step - loss: 1.2939 - val_loss: 1.4125
Epoch 2/10
9375/9375 [==============================] - 6s 644us/step - loss: 0.6974 - val_loss: 1.5488
Epoch 3/10
9375/9375 [==============================] - 6s 659us/step - loss: 0.5811 - val_loss: 1.5663
Epoch 4/10
9375/9375 [==============================] - 6s 659us/step - loss: 0.5209 - val_loss: 1.6327
Epoch 5/10
9375/9375 [==============================] - 6s 664us/step - loss: 0.4784 - val_loss: 1.8563
Epoch 6/10
9375/9375 [==============================] - 6s 665us/step - loss: 0.4405 - val_loss: 1.7205
Epoch 7/10
9375/9375 [==============================] - 6s 660us/step - loss: 0.4116 - val_loss: 1.9137
Epoch 8/10
9375/9375 [==============================] - 6s 669us/step - loss: 0.3898 - val_loss: 1.7058
Epoch 9/10
9375/9375 [==============================] - 6s 659us/step - loss: 0.3745 - val_loss: 1.9133
Epoch 10/10
9375/9375 [==============================] - 6s 668u

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

1172/1172 [==============================] - 0s 290us/step
              precision    recall  f1-score   support

           0       0.48      0.31      0.38      1499
           1       0.36      0.55      0.44      1499
           2       0.46      0.54      0.50      1499
           3       0.65      0.38      0.48      1499
           4       0.88      0.91      0.90      1499
           5       0.72      0.87      0.79      1499
           6       0.78      0.67      0.72      1499
           7       0.93      0.68      0.79      1499
           8       0.64      0.97      0.77      1499
           9       0.82      0.78      0.80      1499
          10       0.90      0.43      0.58      1499
          11       0.83      0.78      0.81      1499
          12       0.76      0.75      0.75      1499
          13       0.30      0.69      0.42      1499
          14       0.35      0.49      0.41      1499
          15       0.94      0.76      0.84      1499
          16       0.7

## 0-32

In [20]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [21]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_3400/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_3400/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_3400/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [22]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [23]:
model3=XGBClassifier(n_estimators=500)
model3.fit(x32,y32,early_stopping_rounds=10, eval_set=[(xv32, yv32)])
y_pred=model3.predict(xt32)
print(classification_report(yt32,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.67182
[1]	validation_0-mlogloss:2.46507
[2]	validation_0-mlogloss:2.32055
[3]	validation_0-mlogloss:2.20414
[4]	validation_0-mlogloss:2.11896
[5]	validation_0-mlogloss:2.03793
[6]	validation_0-mlogloss:1.97623
[7]	validation_0-mlogloss:1.91616
[8]	validation_0-mlogloss:1.86422
[9]	validation_0-mlogloss:1.81560
[10]	validation_0-mlogloss:1.76531
[11]	validation_0-mlogloss:1.73049
[12]	validation_0-mlogloss:1.69274
[13]	validation_0-mlogloss:1.65740
[14]	validation_0-mlogloss:1.62544
[15]	validation_0-mlogloss:1.59777
[16]	validation_0-mlogloss:1.57087
[17]	validation_0-mlogloss:1.54381
[18]	validation_0-mlogloss:1.52039
[19]	validation_0-mlogloss:1.49558
[20]	validation_0-mlogloss:1.47630
[21]	validation_0-mlogloss:1.45636
[22]	validation_0-mlogloss:1.44021
[23]	validation_0-mlogloss:1.41743
[24]	validation_0-mlogloss:1.39870
[25]	validation_0-mlogloss:1.38071
[26]	validation_0-mlogloss:1.36630
[27]	validation_0-mlogloss:1.35541
[28]	validation_0-mlogloss:1.3

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
9375/9375 [==============================] - 7s 711us/step - loss: 0.7572 - val_loss: 1.0026
Epoch 2/10
9375/9375 [==============================] - 7s 699us/step - loss: 0.2478 - val_loss: 0.9522
Epoch 3/10
9375/9375 [==============================] - 7s 708us/step - loss: 0.1755 - val_loss: 0.8248
Epoch 4/10
9375/9375 [==============================] - 7s 709us/step - loss: 0.1439 - val_loss: 0.8511
Epoch 5/10
9375/9375 [==============================] - 7s 699us/step - loss: 0.1243 - val_loss: 0.9427
Epoch 6/10
9375/9375 [==============================] - 7s 708us/step - loss: 0.1116 - val_loss: 0.9574
Epoch 7/10
9375/9375 [==============================] - 7s 699us/step - loss: 0.1031 - val_loss: 0.8959
Epoch 8/10
9375/9375 [==============================] - 7s 695us/step - loss: 0.0962 - val_loss: 0.8762
Epoch 9/10
9375/9375 [==============================] - 7s 715us/step - loss: 0.0884 - val_loss: 0.9605
Epoch 10/10
9375/9375 [==============================] - 7s 698u

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

1172/1172 [==============================] - 0s 290us/step
              precision    recall  f1-score   support

           0       0.82      0.67      0.74      1499
           1       0.64      0.72      0.68      1499
           2       0.86      0.74      0.80      1499
           3       0.72      0.44      0.55      1499
           4       0.96      0.98      0.97      1499
           5       0.97      0.87      0.92      1499
           6       0.93      0.97      0.95      1499
           7       0.95      0.95      0.95      1499
           8       0.72      0.90      0.80      1499
           9       0.86      0.93      0.89      1499
          10       0.94      0.90      0.92      1499
          11       0.95      0.98      0.97      1499
          12       0.79      0.96      0.86      1499
          13       0.77      0.79      0.78      1499
          14       0.67      0.82      0.74      1499
          15       0.93      0.99      0.96      1499
          16       0.9

## 0-64

In [24]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [25]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [26]:
model4=XGBClassifier(n_estimators=500)
model4.fit(x,y,early_stopping_rounds=10, eval_set=[(xv, yv)])
y_pred=model4.predict(xt)
print(classification_report(yt,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.39933
[1]	validation_0-mlogloss:2.13371
[2]	validation_0-mlogloss:1.96682
[3]	validation_0-mlogloss:1.82947
[4]	validation_0-mlogloss:1.71458
[5]	validation_0-mlogloss:1.62135
[6]	validation_0-mlogloss:1.53765
[7]	validation_0-mlogloss:1.46627
[8]	validation_0-mlogloss:1.40357
[9]	validation_0-mlogloss:1.34584
[10]	validation_0-mlogloss:1.29171
[11]	validation_0-mlogloss:1.24807
[12]	validation_0-mlogloss:1.21102
[13]	validation_0-mlogloss:1.16864
[14]	validation_0-mlogloss:1.13320
[15]	validation_0-mlogloss:1.09936
[16]	validation_0-mlogloss:1.06757
[17]	validation_0-mlogloss:1.03871
[18]	validation_0-mlogloss:1.01180
[19]	validation_0-mlogloss:0.98949
[20]	validation_0-mlogloss:0.96749
[21]	validation_0-mlogloss:0.94527
[22]	validation_0-mlogloss:0.92945
[23]	validation_0-mlogloss:0.91330
[24]	validation_0-mlogloss:0.89708
[25]	validation_0-mlogloss:0.88087
[26]	validation_0-mlogloss:0.86842
[27]	validation_0-mlogloss:0.85007
[28]	validation_0-mlogloss:0.8

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
9375/9375 [==============================] - 7s 749us/step - loss: 0.4753 - val_loss: 0.7057
Epoch 2/10
9375/9375 [==============================] - 7s 745us/step - loss: 0.1528 - val_loss: 0.6195
Epoch 3/10
9375/9375 [==============================] - 7s 736us/step - loss: 0.1112 - val_loss: 0.6615
Epoch 4/10
9375/9375 [==============================] - 7s 740us/step - loss: 0.0883 - val_loss: 0.7001
Epoch 5/10
9375/9375 [==============================] - 7s 743us/step - loss: 0.0773 - val_loss: 0.7987
Epoch 6/10
9375/9375 [==============================] - 7s 736us/step - loss: 0.0694 - val_loss: 0.6665
Epoch 7/10
9375/9375 [==============================] - 7s 743us/step - loss: 0.0628 - val_loss: 0.6586
Epoch 8/10
9375/9375 [==============================] - 7s 743us/step - loss: 0.0565 - val_loss: 0.6919
Epoch 9/10
9375/9375 [==============================] - 7s 726us/step - loss: 0.0528 - val_loss: 0.6235
Epoch 10/10
9375/9375 [==============================] - 7s 744u

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

1172/1172 [==============================] - 0s 307us/step
              precision    recall  f1-score   support

           0       0.72      0.83      0.77      1499
           1       0.83      0.94      0.88      1499
           2       0.93      0.75      0.83      1499
           3       0.93      0.87      0.90      1499
           4       0.99      1.00      0.99      1499
           5       0.95      0.87      0.91      1499
           6       0.91      0.98      0.94      1499
           7       0.92      0.92      0.92      1499
           8       0.83      0.93      0.88      1499
           9       0.88      0.78      0.83      1499
          10       0.89      0.76      0.82      1499
          11       0.96      0.94      0.95      1499
          12       0.84      0.97      0.90      1499
          13       0.78      0.89      0.83      1499
          14       0.62      0.87      0.72      1499
          15       0.99      0.99      0.99      1499
          16       0.9